
#### Run the cell below to install the required packages for Copilot


In [ ]:

#Run this cell to install the required packages for Copilot
%load_ext dscopilot_installer
%activate_dscopilot


In [ ]:
# Required Libraries
%pip install openai
%pip install pdf2image


In [ ]:
# Configuration
GPT4V_KEY = "REPLACE_GPT4V_KEY"  # Your GPT4V key
GPT4V_ENDPOINT = "REPLACE_GPT4V_ENDPOINT"  # The API endpoint for your GPT4V instance

In [ ]:
# Core Functionm to send request
from pdf2image import convert_from_path
from datetime import datetime
import os
import requests
import base64
import time
import shutil
import json

def send_request(encoded_images):
    headers = {
        "Content-Type": "application/json",
        "api-key": GPT4V_KEY,
    }

    # Payload for the request
    payload = {
        "enhancements": {
            "ocr": {
                "enabled": True  # enabling OCR to extract text from the image using AI vision services
            },
            "grounding": {
                "enabled": True  # enabling grounding to extract the context of the image using AI vision services
            },
        },
        "messages": [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": """
                            You are a field extraction expert. When given a series of images, extract all the fields into a JSON object structure.
                            Treat the series of documents as one cohesive document and return a json mapping all the appropriate fields.
                            Rewrite json key actual_reading column beginning with the higher month to be called actual_reading_end_reading.
                            Rewrite json key actual_reading column start with the lower month to be called acual_reading_begin_reading
                            Convert all dates to MM-DD-YYY format.
                            Process fields that don't have values as null.
                            Only output the following items in JSON Object.
                            Output json exactly like this:
                  {
    "company": "PSE&G",
    "total_amount_due": "631.78",
    "due_date": "1/1/2021",
    "bill_date": "1/31/2021",
    "billing_period_start": "01/31/2024",
    "billing_period_end": "1/31/2024",
    "account_number": "REDACTED",
    "service_address": "REDACTED",
    "balance_remaining_from_last_bill": "128.28",
    "this_month_charges_and_credits": "103.50",
    "payment_received": "0.00",
    "balance_remaining": "128.28",
    "electric_usage": "6.0% less compared to this month last year",
    "electric_charges": "234.85",
    "electric_actual_reading_begin": "1111",
    "electric_actual_reading_end": "222",
    "electric_usage_difference": "189",
    "electric_service_charge": "1.95",
    "electric_delivery_charges": "2.96",
    "electric_supply_charges": "2.89",
    "electric_total_charges": "100.85",
    "gas_usage": "4.6% less compared to this month last year",
    "gas_actual_begin_reading": "2761",
    "gas_actual_end_reading": "2914",
    "gas_difference": "153",
    "gas_converted_to_ccf": "154.836",
    "gas_total_used_therms": "160.255",
    "gas_monthly_service_charge": "8.62",
    "gas_delivery_charges": "104.95",
    "gas_supply_charges": "63.70",
    "gas_total_charges": "168.65"
}
                        """
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Return the fields in this document as a complete json object",
                    },
                ],
            },
        ],
        "temperature": 0,
        "top_p": 0,
        "max_tokens": 4096,
    }

    # Add an item for each encoded image, limited to 10 images
    for encoded_image in encoded_images[:10]:
        payload["messages"][1]["content"].append({
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"},
        })

    # Send request
    try:
        response = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
        json_response = response.json()
    except requests.RequestException as e:
        raise SystemExit(f"Failed to make the request. Error: {e}")
    return json_response

In [ ]:
# Main Functions

def convert_pdf_to_images(pdf_file_path, image_path):
    # Convert PDF file to images
    images = convert_from_path(pdf_file_path)
    # Save images in new folder
    new_image_folder_path = os.path.join(image_path, f'{os.path.splitext(os.path.basename(pdf_file_path))[0]}_{datetime.now().strftime("%Y%m%d%H%M%S")}')
    os.makedirs(new_image_folder_path)
    for i, image in enumerate(images):
        image.save(os.path.join(new_image_folder_path, f"page{(i+1):02}.jpg"), "JPEG")
    return new_image_folder_path

def encode_images(image_folder_path):
    encoded_images = []
    image_file_paths = [os.path.join(image_folder_path, file) for file in os.listdir(image_folder_path)]
    for image_file_path in image_file_paths:
        encoded_image = base64.b64encode(open(image_file_path, "rb").read()).decode("ascii")
        encoded_images.append(encoded_image)
    return encoded_images

def clean_json_response(response_content):
    # Clean up the response's content. Convert the response's json string to a json object
    # Remove the leading and trailing characters
    json_string = response_content.replace('```json\n', '')
    json_string = json_string.rsplit('\n', 1)[0]
    try:
        # Try to parse the JSON string into a Python dictionary
        json_object = json.loads(json_string)
        print(json_string)
    except json.JSONDecodeError:
        print("The JSON string is not complete.")
    return json_object

def write_json_to_file(pdf_file, json_path, json_object):
    # Create the 'JSON Output' folder 
    new_json_folder_path = os.path.join(json_path, f'{os.path.splitext(pdf_file)[0]}_{datetime.now().strftime("%Y%m%d%H%M%S")}')
    os.makedirs(new_json_folder_path)
    # Write the response to a JSON file in the 'JSON Output' folder
    json_output_filename = os.path.splitext(pdf_file)[0] + ".json"
    json_output_filepath = os.path.join(new_json_folder_path, json_output_filename)
    with open(json_output_filepath, "w") as file:
        json.dump(json_object, file, indent=4)
    return json_output_filepath

def archive_pdf_to_folder(pdf_file, pdf_file_path, archive_folder_path):
    # Create new 'Archive' folder
    new_archive_folder_path = os.path.join(archive_folder_path,f'{os.path.splitext(pdf_file)[0]}_{datetime.now().strftime("%Y%m%d%H%M%S")}')
    os.makedirs(new_archive_folder_path)
   # Move all files from source to destination
    shutil.move(pdf_file_path, new_archive_folder_path)

def load_json_to_table(processsed_json):
    new_path = processsed_json.replace("/lakehouse/default/", "")
    df = spark.read.option("multiLine", "true").json(new_path)
    df.write.mode("append").saveAsTable("bills")




In [ ]:
# Runner
lakehouse_path = "/lakehouse/default/Files"
pdf_folder_path = f"{lakehouse_path}/invoices/" # Path to pdf folder
json_folder_path = f"{lakehouse_path}/json/" # Path to json output folder
image_folder_path = f"{lakehouse_path}/images/" # Path to images
archive_folder_path = f"{lakehouse_path}/processed/" # Path to archive folder
def runner():
    # Get a list of all paths of the pdfs in the pdf folder
    #pdf_paths = [os.path.join(pdf_folder_path, file) for file in os.listdir(pdf_folder_path)]
    # Run the extractor for every pdf in the folder
    for pdf_file in os.listdir(pdf_folder_path):
        # Convert PDF's to Images
        pdf_file_path = os.path.join(pdf_folder_path, pdf_file)
        new_image_folder_path = convert_pdf_to_images(pdf_file_path, image_folder_path)
        # Encode all images in new image folder
        encoded_images = encode_images(new_image_folder_path)
        # Send Request to GPT4V to Return JSON structure from Images
        json_response = send_request(encoded_images)
        # Get the content from the response
        response_content = json_response["choices"][0]["message"]["content"]
        # Clean the response
        json_object = clean_json_response(response_content)
        # Add file name to JSON 
        json_object['file_name'] = pdf_file
        # Output to Output folder directory
        processsed_json = write_json_to_file(pdf_file, json_folder_path, json_object)
        # Load JSON files to table
        load_json_to_table(processsed_json)
        # Move processed pdf to archive folder
        archive_pdf_to_folder(pdf_file, pdf_file_path, archive_folder_path)

# Run Extractor
new_json_folder_paths = runner()
